In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
# Define data generators for train, validation and test sets
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DAP_TEAM1/22A81A0508/MANGO/mango_train-20240216T091504Z-001',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DAP_TEAM1/22A81A0508/MANGO/mango_train-20240216T091504Z-001',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/DAP_TEAM1/22A81A0508/MANGO/predict',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

Found 40 images belonging to 1 classes.
Found 9 images belonging to 1 classes.
Found 0 images belonging to 0 classes.


In [13]:
# Define the model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [14]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
history = model.fit(train_generator,validation_data=val_generator,epochs=5)

Epoch 1/5
2/2 [==============================] - 5s 3s/step - loss: 0.7035 - accuracy: 0.2000 - val_loss: 0.4666 - val_accuracy: 1.0000
Epoch 2/5
2/2 [==============================] - 0s 146ms/step - loss: 0.4620 - accuracy: 1.0000 - val_loss: 0.2331 - val_accuracy: 1.0000
Epoch 3/5
2/2 [==============================] - 0s 211ms/step - loss: 0.1733 - accuracy: 1.0000 - val_loss: 0.0511 - val_accuracy: 1.0000
Epoch 4/5
2/2 [==============================] - 0s 243ms/step - loss: 0.0371 - accuracy: 1.0000 - val_loss: 0.0038 - val_accuracy: 1.0000
Epoch 5/5
2/2 [==============================] - 0s 229ms/step - loss: 0.0039 - accuracy: 1.0000 - val_loss: 1.3684e-04 - val_accuracy: 1.0000


In [17]:
model.save("Model.h5","label.txt")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [18]:
#test your image
from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
import numpy as np
#load the model
model = load_model('/content/Model.h5')
#classes
class_names = ['Raw Mango','Ripen Mango']
data = np.ndarray(shape=(1, 22, 22, 3), dtype=np.float32)

In [19]:
image = Image.open("/content/drive/MyDrive/DAP_TEAM1/22A81A0508/MANGO/predict/Copy of images (17).jpeg").convert("RGB")

# resizing the image to be at least 224x224 and then cropping from the center
size = (22, 22)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

# turn the image into a numpy array
image_array = np.asarray(image)

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

# Load the image into the array
data[0] = normalized_image_array